In [8]:
from transformers import BertTokenizer, TFBertModel
import keras.api._v2.keras

In [ ]:
## How can I apply the embeddings? load and then return the dataframe mythink

In [13]:
def create_text_embeddings(dataset):
    """
    Applies pre-trained BERT embeddings to the feature columns with text data for use within clustering.

    Args:
        dataset (pd.DataFrame) : Dataset containing user interactions.

    Returns:
        dataset (pd.DataFrame) : A dataset with embeddings columns.
    """
    # Initialize the bert tokenizer and model from bert base cased.
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    model = TFBertModel.from_pretrained('bert-base-cased')

    # Create a dense layer with the embedding dimension to process embeddings.
    embedding_dimension = 8
    dense_layer = keras.layers.Dense(embedding_dimension, activation='linear')

    # Define a function to get the embeddings from the models and apply them to the text.
    def get_embeddings(text_1):
        """
        Gets embeddings from pre trained bert model for news information used for clustering.
        """

        # Apply the tokenizer to the text and return it in the tensorflow tensor format. 
        encoded_text = tokenizer(text_1, return_tensors='tf')

        # Get the output from BERT model with the encoded text.
        bert_output = model(encoded_text)

        # Use the pooled output for a single vector representation of the input. 
        pooled_output = bert_output.pooler_output

        # Apply dense layer to project to desired size.
        embedding_vector = dense_layer(pooled_output)
        return embedding_vector

    # Apply the embeddings to the abstract and title columns.
    dataset['abstract_embeddings'] = dataset['abstract'].apply(get_embeddings)
    dataset['title_embeddings'] = dataset['title'].apply(get_embeddings)
    return dataset


IndentationError: expected an indented block (784318394.py, line 5)

In [1]:
import ExploratoryAnalysis.data_processing_modules as dpm
import pandas as pd
import tensorflow as tf
import keras.api._v2.keras as keras
import numpy as np
from sqlalchemy import create_engine

In [4]:
# Starting off we need to load in the dataset and turn it into the form we want
sample_data = pd.read_csv('MIND_small/sample_tf_dataset.csv')
sample_data.drop(columns=['Unnamed: 0'], inplace=True)
sample_data = sample_data[sample_data['abstract'].isna() == False]
AUTOTUNE = tf.data.AUTOTUNE
tf_data_obj = tf.data.Dataset.from_tensor_slices(dict(sample_data))
train_ds = tf_data_obj.cache().prefetch(buffer_size=AUTOTUNE)
text_ds = train_ds.map(lambda x: x['abstract'])

In [ ]:
# Then we need to create a text vectorization layer to handle preprocessing


In [ ]:
test_frame.rename(columns={'time':'time_'}, inplace=True)
test_frame.to_csv('MIND_small/sample_tf_dataset.csv')

def create_connection():
    """
    Creates a connection to the database by taking in a user password.
    """
    database_name = 'user_info'
    connection_user = 'admin'
    try:
        connection_password = input("Enter Password: ")
        connection_host = 'svc-57117697-8d22-448b-8b96-d3b2a46d0970-dml.aws-virginia-6.svc.singlestore.com'
        connection_port = '3306'
        connection_url = f"mysql+pymysql://{connection_user}:{connection_password}@{connection_host}:{connection_port}/{database_name}"
        db_connection = create_engine(connection_url)
        return db_connection
    except Exception as e:
        print(f"Exception {e} occured, try again.\n")
        return create_connection()


def push_to_DB(news : pd.DataFrame, behaviors : pd.DataFrame) -> None:
    """ 
    Pushes data to the SingleStore database for usage in modelling within docker containers and virtual machines.

    Args:
        news (pd.DataFrame) : The dataframe containing the data from the news csv.
        behaviors (pd.DataFrame) : The dataframe containing the data from the behaviors csv.

    Returns:
        None.
    """
    db_connection = create_connection()
    user_interaction_data = dpm.decompose_interactions(num_iterations=100000, news=news, behaviors=behaviors)
    user_interaction_data.to_sql('user_behaviors', if_exists='replace', con=db_connection, index=False)
    print("Push to Database Successful")
